In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append('/data/home/jdavey/notebooks/fastai/mi')
from data.loader import *
from nn.lin import *
from nn.relu import *
from nn.softmaxCrossEntropy import *
from nn.module import Module

In [3]:
import math
import torch

## data_loader

In [4]:
avaliable_urls()

MNIST_URL http://deeplearning.net/data/mnist/mnist.pkl


In [5]:
def normalize(x, m, s): return (x-m)/s
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
n,m = x_train.shape
c = y_train.max()+1
n, m, int(c)

(50000, 784, 10)

In [7]:
class Model(Module):
    def __init__(self, n_in, nh, n_out):
        self.layers = [Lin(n_in,nh),
                       Relu(),
                       Lin(nh,n_out)]
        self.loss = SoftmaxCrossEntropy()
    
    def accuracy(self, pred, yb): return (torch.argmax(pred, dim=1)==yb).float().mean()

In [8]:
model = Model(m,50,int(c))
model

Model(
(layer_0):Linear(in_features=784, out_features=50)
(layer_1):Relu()
(layer_2):Linear(in_features=50, out_features=10)
(layer_3):Softmax() & Cross_entropy_loss()
)

In [9]:
[o.shape for o in model.parameters()]

[torch.Size([784, 50]),
 torch.Size([50]),
 torch.Size([50, 10]),
 torch.Size([10])]

In [10]:
loss = model(x_train, y_train); loss

tensor(-0.0973)

In [11]:
model.backward(y_train)

In [12]:
pred = model.forward(x_train)
model.accuracy(pred, y_train)

tensor(0.0975)

In [13]:
bs = 10000    # batch size
lr = 0.01   # learning rate
epochs = 20# how many epochs to train for

In [14]:
x_train.shape[0]//bs

5

In [15]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        loss = model(xb, yb)
        model.backward(yb)
        
#         for l in model.layers:
#             if hasattr(l, 'w'):
#                 l.w-=l.w.g*lr
#                 l.b-=l.b.g*lr
                
        for parameter in model.parameters():
            parameter-=lr*parameter.g
            
        if i % 5 == 0:
            pred = model.forward(x_valid)
            print(model.accuracy(pred, y_valid))

tensor(0.1156)
tensor(0.2135)
tensor(0.3062)
tensor(0.3964)
tensor(0.4655)
tensor(0.5170)
tensor(0.5544)
tensor(0.5894)
tensor(0.6170)
tensor(0.6385)
tensor(0.6521)
tensor(0.6652)
tensor(0.6749)
tensor(0.6842)
tensor(0.6950)
tensor(0.7055)
tensor(0.7210)
tensor(0.7448)
tensor(0.7725)
tensor(0.7932)
